In [1]:
import pandas as pd
import numpy as np

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220306/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_tripid = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220316/beam/year-2018-iteration-2/ITERS/it.0/"

In [3]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "reason": "category",
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_tripid + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:21: DtypeWarning: Columns (4,17,19,24,25,26,27,40,41,42,45,55,57,58) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 25s


In [4]:
plansSF = pd.read_csv(loc_2018_tripid + '0.plans.csv.gz', compression = 'gzip', dtype = dtypes)

In [5]:
plansSF.sort_values(by = "personId").head()

,tripId,personId,planIndex,planScore,planSelected,planElementType,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,legRouteType,legRouteStartLink,legRouteEndLink,legRouteTravelTime,legRouteDistance,legRouteLinks
1175340,NaN,1,0,277.215965,True,activity,0,Home,605711.160337,4.134402e+06,26748.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1175341,577.0,1,0,277.215965,True,leg,1,NaN,NaN,NaN,NaN,hov3_teleportation,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
1175342,NaN,1,0,277.215965,True,activity,2,othmaint,604273.529654,4.131071e+06,26866.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1175343,578.0,1,0,277.215965,True,leg,3,NaN,NaN,NaN,NaN,hov2_teleportation,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
1175344,NaN,1,0,277.215965,True,activity,4,school,604436.586319,4.135123e+06,54720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [7]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [8]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [9]:
eventsSFCopy = eventsSF.copy()

In [10]:
# shift column 'Name' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [11]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [12]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [13]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [14]:
# Shift column 'Name' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [15]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

Wall time: 1min 35s


In [16]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [17]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [18]:
# Either charging events or just ridehail or transit drivers
IDnan = eventsSF[eventsSF['IDMerged'].isna()]

In [19]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [20]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(6329, 63)

In [21]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

Wall time: 4min 21s


In [22]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [23]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [24]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', 'hov3_teleportation', 'car', nan, 'bike', 'walk', 'car_hov2', 'car_hov3', 'walk_transit', 'drive_transit', 'ride_hail', 'ride_hail_pooled', 'ride_hail_transit', 'bike_transit']


In [25]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', 'hov3_teleportation', 'car', nan, 'bike', 'walk', 'car_hov2', 'car_hov3', 'walk_transit', 'drive_transit', 'ride_hail', 'ride_hail_pooled', 'ride_hail_transit', 'bus', 'tram', 'subway', 'cable_car', 'rail', 'bike_transit']


In [26]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'PersonEntersVehicle', 'TeleportationEvent', 'arrival', 'actstart', 'PathTraversal', 'LeavingParkingEvent', 'PersonLeavesVehicle', 'ParkingEvent', 'PersonCost', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [27]:
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

In [28]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [29]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail', eventsSF['modeBEAM'])

In [30]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [150]:
#eventsSF[eventsSF['modeBEAM'] == "ride_hail_pooled"].head()

In [68]:
sample_car = eventsSF[eventsSF['IDMerged'] == 3]

In [69]:
sample_car.to_csv('C:/Shared-Work/Data/CleanData/sample_car.csv', index = False)

In [ ]:
# eventsSF groupby("IDMerged_num") check if "length"  in row eventsSF['type']=='ModeChoice' == sum(length of eventsSF['type']==

#### Adding new columns

In [31]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [32]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [33]:
eventsSF['travelTime'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [34]:
eventsSF['travelDistance'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [35]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelTime'], np.nan)

In [48]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelDistance'], np.nan)

In [36]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelTime'], np.nan)

In [53]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelDistance'], np.nan)

In [37]:
eventsSF['duration_in_ridehail'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail', eventsSF['travelTime'], np.nan)

In [50]:
eventsSF['distance_ridehail'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail', eventsSF['travelDistance'], np.nan)

In [38]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['travelTime'], np.nan)

In [51]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['travelDistance'], np.nan)

In [39]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelTime'], np.nan)

In [52]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [40]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [70]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [76]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

######

In [ ]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [ ]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Gasoline'), 
                          eventsSF['primaryFuel'], np.nan)

In [ ]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [ ]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

#### Trip Index

In [41]:
eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripIndex.fillna(method='ffill')

In [42]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [43]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

In [ ]:
eventsSF[eventsSF['IDMerged']==181]

In [77]:
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'travelTime': np.sum, 'netCost': np.sum, 'actStartType': np.sum, 'actEndType': np.sum, 'duration_walking': np.sum,
            'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 'duration_in_ridehail': np.sum, 'travelDistance': np.sum,
            'duration_in_transit': np.sum, 'distance_walking': np.sum, 'distance_bike': np.sum, 'distance_ridehail': np.sum, 
            'distance_privateCar': np.sum, 'distance_transit': np.sum, 'legVehicleIds': np.sum, 'fuelFood': np.sum,
            'fuelElectricity': np.sum}).reset_index()

In [78]:
Person_Trip_eventsSF['door_to_door_time'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [79]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['door_to_door_time'] - Person_Trip_eventsSF['travelTime'] 

In [80]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [81]:
Person_Trip_eventsSF.head(500).to_csv('C:/Shared-Work/Data/CleanData/pivot_check.csv', index = False)   

In [108]:
sample_bike2 = eventsSF[eventsSF['IDMerged'] == "3550481"]

In [116]:
sample_bike2 = eventsSF[(eventsSF['primaryFuelType'] == "Gasoline") & (eventsSF['modeBEAM'] == "bike") ]

In [120]:
sample_bike2.head()

,IDMerged,person,driver,riders,vehicleType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicle,actType,time,type,link,legMode,primaryFuelLevel,secondaryFuelLevel,facility,score,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,reason,links,numPassengers,primaryFuel,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,incentive,tollCost,netCost,departTime,cost
248,468282,NaN,468282,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49567,NaN,18146.0,PathTraversal,NaN,NaN,2.203726e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,591.851,NaN,NaN,NaN,NaN,NaN,NaN,"71796,71799,29023,13743,13742",0.0,6273.6206,NaN,NaN,2.0,0.0,37.337513,-122.057953,37.337514,-122.060283,2.0,18146.0,18028.0,"27.254,13.274,0.638,52.229,52.229",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
316,2925369,NaN,2925369,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,734823,NaN,18095.0,PathTraversal,NaN,NaN,2.205912e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,385.622,NaN,NaN,NaN,NaN,NaN,NaN,"10188,10189",0.0,4087.5932,NaN,NaN,2.0,0.0,37.967986,-121.785963,37.967986,-121.785963,2.0,18095.0,18018.0,"77.124,77.124",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
16062,2643355,NaN,2643355,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202653,NaN,18250.0,PathTraversal,NaN,NaN,2.197108e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,1216.258,NaN,NaN,NaN,NaN,NaN,NaN,"68460,82455,69029,69022",0.0,12892.3348,NaN,NaN,2.0,0.0,37.839078,-122.275288,37.828203,-122.275229,2.0,18250.0,18007.0,"67.23,7.505,56.199,179.548",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
18520,2008662,NaN,2008662,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737344,NaN,18291.0,PathTraversal,NaN,NaN,2.209720e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,26.422,NaN,NaN,NaN,NaN,NaN,NaN,"162585,162584",0.0,280.0732,NaN,NaN,2.0,0.0,37.473750,-122.232240,37.473750,-122.232240,2.0,18291.0,18286.0,"5.284,5.284",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
26940,3707112,NaN,3707112,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52636,NaN,18416.0,PathTraversal,NaN,NaN,2.206459e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,334.028,NaN,NaN,NaN,NaN,NaN,NaN,"35377,35376",0.0,3540.6968,NaN,NaN,2.0,0.0,37.502374,-122.270943,37.502374,-122.270943,2.0,18416.0,18349.0,"66.806,66.806",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN


In [109]:
sample_bike2.to_csv('C:/Shared-Work/Data/CleanData/sample_bike2.csv', index = False)

#### Merging with activity sim persons and housholds files

In [ ]:
actloc_2018 =  "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220306/activitysim/"

households = pd.read_csv(actloc_2018 + 'final_households.csv')
persons = pd.read_csv(actloc_2018 + 'final_persons.csv')